In [ ]:
%load_ext tensorboard # запускает боард в колаб

import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras import utils

'''Модель классификации текста для анализа в TensorBoard'''

# количество слов, рассматриваемых как признаки
max_features = 2000
# обрезка текста после этого количества слов
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = utils.data_utils.pad_sequences(x_train, maxlen=max_len)
x_test = utils.data_utils.pad_sequences(x_test, maxlen=max_len)

model = keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

%tensorboard mkdir my_log_dir

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir='logs', # файлы журналов будут сохр в этой папке
        histogram_freq=1, # запись гистограммы в к. эпохе
        embeddings_freq=1, # запись векторных представлений в к. эпохе
    )
]
history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)	

%tensorboard --logdir=my_log_dir #  показывает боард в коллабе

# представление в виде графов слоев, созданное plot_model
from keras.utils import plot_model

plot_model(model, to_file='model.png')

# изображение модели с информацией о форме слоев
from keras.utils import plot_model

plot_model(model, show_shapes=True, to_file='model.png')